In [1]:
### CBC SCRAPING CODE
### Authors: JONATHAN CHAN and PANDRAMISHI NAGA SIRISHA

###MOST RECENT UPDATE:  
##2020 MAY 13, 11:52AM
#wrote separate functions for each JSON element to be collected
#extract_json_items() will run for all articles, and will return null if not in proper format

#TO DO:
#write code for collecting JSON items from all articles returned in CBC API (code cell 4)
#write code to iterate through multiple pages of API call 
#done
#write code to store JSON in a JSON or text document
#Final run: collect 50-100 articles for each of 6 indicators

In [19]:
import urllib.request
import json 
from bs4 import BeautifulSoup
#from datetime import date

#import parser if we want to export datetime objects to JSON - currently causing an error
#from dateutil import parser


In [47]:
#https://www.cbc.ca/search_api/v1/search?q=mortgage%20rate&sortOrder=relevance&page=100&fields=feed


def get_initial_url(search_term):
    """returns the URL of the first page API call given a search string"""
    
    words = search_term.split()
    url_prefix = "https://www.cbc.ca/search_api/v1/search?"
#     url_prefix = "https://www.cbc.ca/search/v1/search?"

    query = "q=" + "%20".join(words)
    url_suffix = "&sortOrder=relevance&page=1&fields=feed"
    
    first_url = url_prefix + query + url_suffix
    
    print("FIRST URL API CALL: ", first_url)
    
    return first_url
    
    
get_initial_url("interest rate index")

FIRST URL API CALL:  https://www.cbc.ca/search_api/v1/search?q=interest%20rate%20index&sortOrder=relevance&page=1&fields=feed


'https://www.cbc.ca/search_api/v1/search?q=interest%20rate%20index&sortOrder=relevance&page=1&fields=feed'

In [48]:
first_url = get_initial_url("interest rate")
with urllib.request.urlopen(first_url) as url:
    data = json.loads(url.read().decode())
    for article_dict in data:
        print(article_dict["url"])
        article_url = url
        print("----")


FIRST URL API CALL:  https://www.cbc.ca/search_api/v1/search?q=interest%20rate&sortOrder=relevance&page=1&fields=feed
//www.cbc.ca/news/business/powel-trump-negative-rates-1.5567512
----
//www.cbc.ca/news/canada/british-columbia/vancity-ceo-tamara-vrooman-vancouver-airport-authority-1.5568224
----
//www.cbc.ca/player/play/1707273795558
----
//www.cbc.ca/player/play/1706560067565
----
//www.cbc.ca/news/canada/british-columbia/programs/onthecoast/march-5-2020-interest-rates-1.5487903
----
//www.cbc.ca/player/play/1707557955587
----
//www.cbc.ca/news/canada/ottawa/programs/allinaday/money-column-interest-rate-cut-1.5486004
----
//www.cbc.ca/player/play/1706421827523
----
//www.cbc.ca/news/business/interest-rates-coronavirus-evans-1.5515884
----
//www.cbc.ca/news2/interactives/interest-rates/
----


In [22]:
import requests
import json

In [79]:
def get_and_print_data(url):
    count = 0
    main_url = url
    r = requests.get(url)
    info = r.json()
    last_retrieved_items_count= len(info)
    
    for i in info:
        count += 1
        
    page_number = 2
    
    while last_retrieved_items_count > 0:
        split_url = main_url.split('page')
        new_url = split_url[0] + "page=" + str(page_number) + "&fields=feed" 
        r = requests.get(new_url)
        info = r.json()
        
        last_retrieved_items_count= len(info)
#         print("last_retrieved_items_count", last_retrieved_items_count)

        for i in info:
            count += 1
        page_number += 1    
        print(count)


first_url = get_initial_url("interest rate")
get_and_print_data(first_url)

FIRST URL API CALL:  https://www.cbc.ca/search_api/v1/search?q=interest%20rate&sortOrder=relevance&page=1&fields=feed
last_retrieved_items_count 10
20
last_retrieved_items_count 10
30
last_retrieved_items_count 10
40
last_retrieved_items_count 10
50
last_retrieved_items_count 10
60
last_retrieved_items_count 10
70
last_retrieved_items_count 10
80
last_retrieved_items_count 10
90
last_retrieved_items_count 10
100
last_retrieved_items_count 10
110
last_retrieved_items_count 10
120
last_retrieved_items_count 10
130
last_retrieved_items_count 10
140
last_retrieved_items_count 10
150
last_retrieved_items_count 10
160
last_retrieved_items_count 10
170
last_retrieved_items_count 10
180
last_retrieved_items_count 10
190
last_retrieved_items_count 10
200
last_retrieved_items_count 10
210
last_retrieved_items_count 10
220
last_retrieved_items_count 10
230
last_retrieved_items_count 10
240
last_retrieved_items_count 10
250
last_retrieved_items_count 10
260
last_retrieved_items_count 10
270
last_r

last_retrieved_items_count 10
2360
last_retrieved_items_count 10
2370
last_retrieved_items_count 10
2380
last_retrieved_items_count 10
2390
last_retrieved_items_count 10
2400
last_retrieved_items_count 10
2410
last_retrieved_items_count 10
2420
last_retrieved_items_count 10
2430
last_retrieved_items_count 10
2440
last_retrieved_items_count 10
2450
last_retrieved_items_count 10
2460
last_retrieved_items_count 10
2470
last_retrieved_items_count 10
2480
last_retrieved_items_count 10
2490
last_retrieved_items_count 10
2500
last_retrieved_items_count 10
2510
last_retrieved_items_count 10
2520
last_retrieved_items_count 10
2530
last_retrieved_items_count 10
2540
last_retrieved_items_count 10
2550
last_retrieved_items_count 10
2560
last_retrieved_items_count 10
2570
last_retrieved_items_count 10
2580
last_retrieved_items_count 10
2590
last_retrieved_items_count 10
2600
last_retrieved_items_count 10
2610
last_retrieved_items_count 10
2620
last_retrieved_items_count 10
2630
last_retrieved_items

last_retrieved_items_count 10
4710
last_retrieved_items_count 10
4720
last_retrieved_items_count 10
4730
last_retrieved_items_count 10
4740
last_retrieved_items_count 10
4750
last_retrieved_items_count 10
4760
last_retrieved_items_count 10
4770
last_retrieved_items_count 10
4780
last_retrieved_items_count 10
4790
last_retrieved_items_count 10
4800
last_retrieved_items_count 10
4810
last_retrieved_items_count 10
4820
last_retrieved_items_count 10
4830
last_retrieved_items_count 10
4840
last_retrieved_items_count 10
4850
last_retrieved_items_count 10
4860
last_retrieved_items_count 10
4870
last_retrieved_items_count 10
4880
last_retrieved_items_count 10
4890
last_retrieved_items_count 10
4900
last_retrieved_items_count 10
4910
last_retrieved_items_count 10
4920
last_retrieved_items_count 10
4930
last_retrieved_items_count 10
4940
last_retrieved_items_count 10
4950
last_retrieved_items_count 10
4960
last_retrieved_items_count 10
4970
last_retrieved_items_count 10
4980
last_retrieved_items

last_retrieved_items_count 10
7060
last_retrieved_items_count 10
7070
last_retrieved_items_count 10
7080
last_retrieved_items_count 10
7090
last_retrieved_items_count 10
7100
last_retrieved_items_count 10
7110
last_retrieved_items_count 10
7120
last_retrieved_items_count 10
7130
last_retrieved_items_count 10
7140
last_retrieved_items_count 10
7150
last_retrieved_items_count 10
7160
last_retrieved_items_count 10
7170
last_retrieved_items_count 10
7180
last_retrieved_items_count 10
7190
last_retrieved_items_count 10
7200
last_retrieved_items_count 10
7210
last_retrieved_items_count 10
7220
last_retrieved_items_count 10
7230
last_retrieved_items_count 10
7240
last_retrieved_items_count 10
7250
last_retrieved_items_count 10
7260
last_retrieved_items_count 10
7270
last_retrieved_items_count 10
7280
last_retrieved_items_count 10
7290
last_retrieved_items_count 10
7300
last_retrieved_items_count 10
7310
last_retrieved_items_count 10
7320
last_retrieved_items_count 10
7330
last_retrieved_items

In [24]:
# #OBSOLETE - CANNOT DEAL WITH CASES WHERE JSON DICT ITEMS CANNOT BE FOUND
# def extract_json_items(url, specify_source_type=True):
#     """Returns a json containing the following items from a CBC article:
#         url: the url of the article
#         urlToImage: the url of the header image
#         title: the title of the article 
#         description: subheader of the article
#         author: author (note that some articles do not specify author)
#         source: CBC if specify_source_type == False, subdivision of CBC if True (ex: "CBC radio")
#         publishedAt: tuple of (date_string, datetime object)
        
#         input: url returned from CBC API in "url" field (missing "http:" as part of URL)
#     """
#     json_dict = {}
#     article_url = "http:" + url
    
#     #get HTML from article URL into BeautifulSoup
#     html_bytes = urllib.request.urlopen(article_url)
#     mybytes = html_bytes.read()
#     html = mybytes.decode("utf8")
#     html_bytes.close()
#     soup = BeautifulSoup(html, 'html.parser')

#     #TITLE in detailHeadline (h1 tag)
#     title_h1 = soup.find("h1", {"class": "detailHeadline"})
#     #print("TITLE: ", title_h1.text)
#     json_dict["title"] = title_h1.text

#     #DESCRIPTION IN deck (h2 tag)
#     desc_h2 = soup.find("h2", {"class": "deck"})
#     #print("DESCRIPTION: ", desc_h2.text)
#     json_dict["description"] = desc_h2.text


#     #AUTHOR in authorText span
#     author_span = soup.find("span", {"class": "authorText"})
#     print(author_span)
#     if author_span:
#         json_dict["author"] = author_span.text
#     else:
#         json_dict["author"] = None


#     #source appears before <span class="bullet"> · </span>
#     #if author is attached, there are two bullet tags
#     #if no author attached, there is one bullet tag
#     if specify_source_type:
#         bullet_spans = soup.find_all("span", {"class": "bullet"})
#         for bullet_span in bullet_spans:
#             previous_str = str(bullet_span.previous_sibling)
#             if previous_str.startswith("CBC"):
#                 source = previous_str
#     else:
#         source = "CBC"
#     json_dict["source"] = source

#     time_tag = soup.find("time", {"class": "timeStamp"})
#     datetime_str = time_tag.attrs["datetime"]
#     datetime_obj = parser.isoparse(datetime_str)
    
#     time_str = time_tag.text.split("|")[0].replace("Posted: ", "").strip()
    
#     #ERROR: if we want to return a datetime object, error when writing to JSON
#     #json_dict["publishedAt"] = (time_str, datetime_obj)
#     #write as string for now
#     json_dict["publishedAt"] = (time_str, datetime_str)


#     main_image_tag = soup.find("figure", {"class": "imageMedia leadmedia-story full"})
#     main_image_url = main_image_tag.find("img").attrs["src"]
#     json_dict["urlToImage"] = main_image_url
    
#     json_dict["url"] = article_url

#     #print("ARTICLE URL: ", article_url)
# #     for key, item in json_dict.items():
# #         print(key + ":")
# #         print(item)
# #         print("-----")
    
#     final_json = json.dumps(json_dict)
#     return final_json

       

In [25]:
# #test for separate functions 
# test_url = "//www.cbc.ca/news/politics/federal-deficit-higher-than-252-billion-1.5566768"
# #test_url = "//www.cbc.ca/news/canada/coronavirus-covid19-world-canada-may12-1.5564261"
# article_url = "http:" + test_url

# #get HTML from article URL into BeautifulSoup
# html_bytes = urllib.request.urlopen(article_url)
# mybytes = html_bytes.read()
# html = mybytes.decode("utf8")
# html_bytes.close()
# soup = BeautifulSoup(html, 'html.parser')

In [49]:
def get_author(soup):
    """returns the author of a BeautifulSoup article if it exists, None if cannot be found
    
    Assume author info is contained within span tag (class: authorText)
    """
    author_span = soup.find("span", {"class": "authorText"})
    
    if author_span:
        return author_span.text
    else:
        #print("No author found in article!")
        return None

get_author(soup)

NameError: name 'soup' is not defined

In [27]:
def get_title(soup):
    """returns the title of a BeautifulSoup article if it exists, None if cannot be found
    
    Assume title info is contained within h1 tag (class: detailHeadline)
    """
    title_tag = soup.find("h1", {"class": "detailHeadline"})
    
    if title_tag:
        title_text = title_tag.text
        return title_text
    else:
        #print("no title found in article!")
        return None

    

In [28]:
def get_desc(soup):
    """returns the description of a BeautifulSoup article if it exists, None if not
    
    Assume description is contained within h2 tag (class: deck)
    """
    desc_tag = soup.find("h2", {"class": "deck"})
    
    if desc_tag:
        desc_text = desc_tag.text
        return desc_text
    else:
        #print("No description found in article!")
        return None

get_desc(soup)
    

NameError: name 'soup' is not defined

In [29]:
def get_url_to_image(soup):
    """returns the url to the header image of a CBC article (BeautifulSoup) if it exists, None if not
    
    Assume image url is contained within src attribute of img tag 
    """
    main_image_tag = soup.find("figure", {"class": "imageMedia leadmedia-story full"})
    
    if main_image_tag:
        main_image_url = main_image_tag.find("img").attrs["src"]
        return main_image_url
    else:
        #print("No main header image found in article!")
        return None
    
get_url_to_image(soup)
    

NameError: name 'soup' is not defined

In [30]:
def get_publish_time(soup):
    """returns a tuple of publish time string and datetime string if found in article, None if not
    
    Assume time is contained within time tag (class: timestamp)
    """
    time_tag = soup.find("time", {"class": "timeStamp"})
    if time_tag:
        datetime_str = time_tag.attrs["datetime"]
        
        #NOTE: if we want to return a datetime object, error when writing to JSON
        #datetime_obj = parser.isoparse(datetime_str)
        #SOLUTION: return as string for now, convert to datetime object later in pipeline
        
        #format of time_tag.text: 
        timetext_str = time_tag.text.split("|")[0].replace("Posted: ", "").strip()
        return (timetext_str, datetime_str)
    else:
        #print("No time information found in article!")
        return None
get_publish_time(soup)

NameError: name 'soup' is not defined

In [34]:
def get_source(soup, specify_source_type=True):
    """Returns the source of the article if it exists
    if specify_source_type, subdivision of CBC will be returned
    if not, "CBC" will be returned as the source
    
    
    Assume that source always starts with "CBC" (Ex: "CBC news", "CBC radio")
    Assume that source comes before span tag (class: bullet)
    """
    
    #source appears before <span class="bullet"> · </span>
    #if author is attached, there are two bullet tags
    #if no author attached, there is one bullet tag
    source = None
    
    if specify_source_type:
        bullet_spans = soup.find_all("span", {"class": "bullet"})
        for bullet_span in bullet_spans:
            previous_str = str(bullet_span.previous_sibling)
            if previous_str.startswith("CBC"):
                source = previous_str
    else:
        
        source = "CBC"
    
    if source:
        return source
    else:
        #print("no source found in article!")
        return None
    

get_source(soup)   

NameError: name 'soup' is not defined

In [32]:
#NEW - USING NEW FUNCTIONS: 
def extract_json_items(url, specify_source_type=True):
    """Returns a json containing the following items from a CBC article:
        url: the url of the article
        urlToImage: the url of the header image
        title: the title of the article 
        description: subheader of the article
        author: author (note that some articles do not specify author)
        source: CBC if specify_source_type == False, subdivision of CBC if True (ex: "CBC radio")
        publishedAt: tuple of (date_string, datetime object)
        
        input: url returned from CBC API in "url" field (missing "http:" as part of URL)
    """
    json_dict = {}
    article_url = "http:" + url
    
    #get HTML from article URL into BeautifulSoup
    html_bytes = urllib.request.urlopen(article_url)
    mybytes = html_bytes.read()
    html = mybytes.decode("utf8")
    html_bytes.close()
    soup = BeautifulSoup(html, 'html.parser')
    
    author_name = get_author(soup)
    title_text = get_title(soup)
    desc_text = get_desc(soup)
    image_url = get_url_to_image(soup)
    publish_time = get_publish_time(soup)
    news_source = get_source(soup)

    json_dict["author"] = author_name
    json_dict["title"] = title_text 
    json_dict["description"] = desc_text
    json_dict["url"] = article_url
    json_dict["urlToImage"] = image_url
    json_dict["publishedAt"] = publish_time
    json_dict["source"] = news_source
    
    print("JSON KEYS AND VALUES: ")
    for key, item in json_dict.items():
        print(key + ": " + str(item))
    print("-----")
    
    final_json = json.dumps(json_dict)
    return final_json


In [33]:
#TEST ON DIFFERENT CBC LINKS RETURNED BY CBC API

standard_url = "//www.cbc.ca/news/politics/federal-deficit-higher-than-252-billion-1.5566768"
radio_url = "//www.cbc.ca/radio/costofliving/slashed-interest-rates-getting-a-piece-of-the-electric-car-pie-and-a-happy-jobs-friday-to-all-1.5486253"
media_url = "//www.cbc.ca/player/play/1707317315674"
noauthor_url = "//www.cbc.ca/news/canada/coronavirus-covid19-world-canada-may12-1.5564261"



extract_json_items(standard_url)

#note: doesn't work for 'player' URLS ("//www.cbc.ca/player/play/1707317315674")
#will run, but will return null for most values - player articles have different setup


JSON KEYS AND VALUES: 
author: John Paul Tasker
title: Federal deficit likely to be higher than $252 billion, parliamentary budget officer says
description: PBO says it's possible federal debt will hit $1 trillion because of pandemic relief spending
url: http://www.cbc.ca/news/politics/federal-deficit-higher-than-252-billion-1.5566768
urlToImage: https://i.cbc.ca/1.5138746.1558027091!/fileImage/httpImage/image.jpg_gen/derivatives/16x9_780/phoenix-pay-system.jpg
publishedAt: ('May 12, 2020 5:34 PM ET', '2020-05-13T17:44:54.995Z')
source: CBC News
-----


'{"author": "John Paul Tasker", "title": "Federal deficit likely to be higher than $252 billion, parliamentary budget\\u00a0officer says", "description": "PBO says it\'s possible federal debt will hit $1 trillion because of pandemic relief spending", "url": "http://www.cbc.ca/news/politics/federal-deficit-higher-than-252-billion-1.5566768", "urlToImage": "https://i.cbc.ca/1.5138746.1558027091!/fileImage/httpImage/image.jpg_gen/derivatives/16x9_780/phoenix-pay-system.jpg", "publishedAt": ["May 12, 2020 5:34 PM ET", "2020-05-13T17:44:54.995Z"], "source": "CBC News"}'